In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\crvid\AppData\Local\Temp\ipykernel_2668\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data_train = pd.read_csv("C:\\Users\\crvid\\Documents\\IRONHACK\\WEEK_7\\DAY1\\lab-natural-language-processing\\data\\kg_train.csv",encoding='latin-1')
data_test = pd.read_csv("C:\\Users\\crvid\\Documents\\IRONHACK\\WEEK_7\\DAY1\\lab-natural-language-processing\\data\\kg_test.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data_train = data_train.head(1000)
print(data_train.shape)
data_train.fillna("",inplace=True)

data_test = data_test.head(1000)
print(data_test.shape)
data_test.fillna("",inplace=True)

(1000, 2)
(1000, 1)


### Let's divide the training and test set into two partitions

In [4]:
from sklearn.model_selection import train_test_split

# Defining Features and Target
X = data_train["text"]
y = data_train["label"]

# Splitting the data_train into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

## Data Preprocessing

In [5]:
import re
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english') # Initializes the stemmer

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [6]:
def clean_text(text):
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL) # Removes everything inside <script> and <style> tags
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL) # Removes HTML comments, anything in between <!-- and -->
    text = re.sub(r'<.*?>', '', text) # Removes regular HTML tags
    return text

# Apply the clean_text function to the X_train and X_test datasets
X_train_clean = X_train.apply(clean_text)
X_test_clean = X_test.apply(clean_text)

# Check the cleaned data (just to see a small sample)
print(X_train_clean.head())
print(X_test_clean.head())

56     Tom harkin called office twice duringCell:Home:AM
955    Dear sir=2C I got your contact in cause of a s...
231                                                   18
738                               She is on her cell.###
740                         Pls keep the updates coming!
Name: text, dtype: object
518     DR,GAIUS OBASEKIFORMER GROUP DIRECTORNIGERIA ...
871    R29vZCBEYXksDQpNYXkgaXQgbm90IGJlIGEgc3VycHJpc2...
797    newmyer on behalf of Jackie NewmyerTuesday Sep...
274    GREETINGS FROM MR. ABU M.SESAYDEAR FRIEND,THRO...
325    FROM:RITA DOINK.INTERNATIONAL PROMOTIONS/PRIZE...
Name: text, dtype: object


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [7]:
# Let's expand the clean_text function to include all the additional cleaning steps

def clean_text(text):
    # Remove inline JavaScript/CSS and HTML tags
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    text = re.sub(r'<.*?>', '', text)
    
    # Remove special characters (non-alphabetic characters except space)
    text = re.sub(r'[^A-Za-z\s]', ' ', text)
    
    # Remove numbers
    text = re.sub(r'\d+', ' ', text)
    
    # Remove single characters
    text = re.sub(r'\b\w\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove prefixed 'b' (sometimes added when converting byte data)
    text = re.sub(r'^b\s+', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

In [8]:
# Apply the updated clean_text function to the X_train and X_test datasets
X_train_clean = X_train.apply(clean_text)
X_test_clean = X_test.apply(clean_text)

# Check the cleaned data (just to see a small sample)
print(X_train_clean.head())
print(X_test_clean.head())

56     tom harkin called office twice duringcell home am
955    dear sir got your contact in cause of seriouse...
231                                                     
738                                   she is on her cell
740                          pls keep the updates coming
Name: text, dtype: object
518    dr gaius obasekiformer group directornigeria n...
871    vzcbeyxksdqpnyxkgaxqgbm igjligegc vychjpc ugdg...
797    newmyer on behalf of jackie newmyertuesday sep...
274    greetings from mr abu sesaydear friend through...
325    from rita doink international promotions prize...
Name: text, dtype: object


## Now let's work on removing stopwords
Remove the stopwords.

In [9]:
# Let's expand the clean_text function to remove stopwords

def clean_text(text):
    # Remove inline JavaScript/CSS and HTML tags
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    text = re.sub(r'<.*?>', '', text)
    
    # Remove special characters (non-alphabetic characters except space)
    text = re.sub(r'[^A-Za-z\s]', ' ', text)
    
    # Remove numbers
    text = re.sub(r'\d+', ' ', text)
    
    # Remove single characters
    text = re.sub(r'\b\w\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove prefixed 'b' (sometimes added when converting byte data)
    text = re.sub(r'^b\s+', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = " ".join([word for word in text.split() if word not in stop_words])

    return text

In [10]:
# Apply the updated clean_text function to both X_train and X_test
X_train_clean = X_train.apply(clean_text)
X_test_clean = X_test.apply(clean_text)

# Check the cleaned data (after removing stopwords)
print(X_train_clean.head())
print(X_test_clean.head())

56        tom harkin called office twice duringcell home
955    dear sir got contact cause seriouse search rel...
231                                                     
738                                                 cell
740                              pls keep updates coming
Name: text, dtype: object
518    dr gaius obasekiformer group directornigeria n...
871    vzcbeyxksdqpnyxkgaxqgbm igjligegc vychjpc ugdg...
797    newmyer behalf jackie newmyertuesday september...
274    greetings mr abu sesaydear friend courtesy bus...
325    rita doink international promotions prize awar...
Name: text, dtype: object


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [11]:
import nltk
from nltk.stem import WordNetLemmatizer

# Download WordNet data if not already downloaded
nltk.download('wordnet')
nltk.download('omw-1.4')  # To get additional support for languages

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\crvid\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\crvid\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [12]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Remove inline JavaScript/CSS and HTML tags
    text = re.sub(r'<(script|style).*?>.*?</\1>', '', text, flags=re.DOTALL)
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    text = re.sub(r'<.*?>', '', text)
    
    # Remove special characters (non-alphabetic characters except space)
    text = re.sub(r'[^A-Za-z\s]', ' ', text)
    
    # Remove numbers
    text = re.sub(r'\d+', ' ', text)
    
    # Remove single characters
    text = re.sub(r'\b\w\b', '', text)
    
    # Remove single characters from the start
    text = re.sub(r'^[a-zA-Z]\s+', '', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in text.split() if word not in stop_words]
    
    # Lemmatization (convert words to their base form)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    
    # Reconstruct the cleaned text
    text = ' '.join(lemmatized_words)
    
    return text

In [13]:
# Apply the updated clean_text function with lemmatization to X_train and X_test
X_train_clean = X_train.apply(clean_text)
X_test_clean = X_test.apply(clean_text)

# Check the cleaned and lemmatized data
print(X_train_clean.head())
print(X_test_clean.head())

56        tom harkin called office twice duringcell home
955    dear sir got contact cause seriouse search rel...
231                                                     
738                                                 cell
740                               pls keep update coming
Name: text, dtype: object
518    dr gaius obasekiformer group directornigeria n...
871    vzcbeyxksdqpnyxkgaxqgbm igjligegc vychjpc ugdg...
797    newmyer behalf jackie newmyertuesday september...
274    greeting mr abu sesaydear friend courtesy busi...
325    rita doink international promotion prize award...
Name: text, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [42]:
from collections import Counter

# Separate ham and spam messages
ham_messages = data_train[data_train['label'] == 0]['text']
spam_messages = data_train[data_train['label'] == 1]['text']

# Clean the messages with out created function
ham_cleaned = ham_messages.apply(clean_text)
spam_cleaned = spam_messages.apply(clean_text)

# Tokenize the words by splitting the cleaned text into words
ham_words = " ".join(ham_cleaned).split()
spam_words = " ".join(spam_cleaned).split()

# Count word frequency using Counter from the collections module
ham_word_count = Counter(ham_words)
spam_word_count = Counter(spam_words)

# Get the 10 most common words in ham and spam messages
top_10_ham = ham_word_count.most_common(10)
top_10_spam = spam_word_count.most_common(10)

# Display the results
print("Top 10 words in HAM messages:")
for word, count in top_10_ham:
    print(f"{word}: {count}")

print("Top 10 words in SPAM messages:")
for word, count in top_10_spam:
    print(f"{word}: {count}")

Top 10 words in HAM messages:
state: 136
pm: 127
would: 107
president: 99
time: 95
call: 94
mr: 91
obama: 84
percent: 81
secretary: 79
Top 10 words in SPAM messages:
money: 981
account: 895
bank: 800
fund: 781
u: 734
transaction: 549
business: 514
country: 508
mr: 489
nbsp: 475


## Extra features

In [40]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,money_mark,suspicious_words,text_len
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,1,0,2292
1,Will do.,0,1,0,8
2,Nora--Cheryl has emailed dozens of memos about...,0,1,0,197
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,1,1,2199
4,fyi,0,1,0,3


## How would work the Bag of Words with Count Vectorizer concept?

In [51]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit the CountVectorizer on the training data and transform it into a Bag of Words
X_train_bow = vectorizer.fit_transform(X_train_clean)

# Convert the BoW matrix to a DataFrame for easier inspection
bow_df = pd.DataFrame(X_train_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Check the shape and a few samples of the Bag of Words representation
print(f"Shape of the Bag of Words matrix: {X_train_bow.shape}")
print(bow_df.head())


Shape of the Bag of Words matrix: (800, 27204)
   aa  aaa  aabeiawaeaambiqaceqedeqh  aac  aaccount  aacute  aacw  aae  \
0   0    0                         0    0         0       0     0    0   
1   0    0                         0    0         0       0     0    0   
2   0    0                         0    0         0       0     0    0   
3   0    0                         0    0         0       0     0    0   
4   0    0                         0    0         0       0     0    0   

   aaecaxeebsexbhjbuqdhcrmimoeifekrobhbcsmzuvavynlrchyknoel  aaegmdbsch  ...  \
0                                                  0                  0  ...   
1                                                  0                  0  ...   
2                                                  0                  0  ...   
3                                                  0                  0  ...   
4                                                  0                  0  ...   

   zzvpscw  zzwh  zzwqgb  z

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit the vectorizer on the cleaned training data and transform it into TF-IDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_clean)

# Check the shape of the vectorized dataset (number of messages and features)
print(f"Shape of the TF-IDF matrix: {X_train_tfidf.shape}")

# Optionally, print the first few feature names (words) and their corresponding TF-IDF values for the first message
print("First message's TF-IDF representation:")
print(pd.DataFrame(X_train_tfidf[0].toarray(), columns=tfidf_vectorizer.get_feature_names_out()).head())

Shape of the TF-IDF matrix: (800, 27204)
First message's TF-IDF representation:
    aa  aaa  aabeiawaeaambiqaceqedeqh  aac  aaccount  aacute  aacw  aae  \
0  0.0  0.0                       0.0  0.0       0.0     0.0   0.0  0.0   

   aaecaxeebsexbhjbuqdhcrmimoeifekrobhbcsmzuvavynlrchyknoel  aaegmdbsch  ...  \
0                                                0.0                0.0  ...   

   zzvpscw  zzwh  zzwqgb  zzwqgdg  zzwqgyw  zzwx  zzxh  zzz  zzzahbxntxe  zzzj  
0      0.0   0.0     0.0      0.0      0.0   0.0   0.0  0.0          0.0   0.0  

[1 rows x 27204 columns]


## And the Train a Classifier?

In [56]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Initialize the classifier
nb_classifier = MultinomialNB()

# Step 2: Train the classifier using the training data
# X_train_tfidf contains the TF-IDF vectorized training messages
# y_train contains the labels (ham/spam)
nb_classifier.fit(X_train_tfidf, y_train)

# Step 3: Transform the test data using the same TF-IDF vectorizer
# X_test_clean is your cleaned test set (from text preprocessing)
X_test_tfidf = tfidf_vectorizer.transform(X_test_clean)

# Step 4: Predict the labels for the test data
y_pred = nb_classifier.predict(X_test_tfidf)

# Step 5: Evaluate the classifier performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Detailed classification report (precision, recall, f1-score)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 92.50%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.87      0.93       117
           1       0.85      1.00      0.92        83

    accuracy                           0.93       200
   macro avg       0.92      0.94      0.92       200
weighted avg       0.94      0.93      0.93       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code